In [5]:
import fastapi
import requests
from dotenv import load_dotenv
import os

# gets all existing weapons info
def get_schema():
    schema = {}
    load_dotenv()
    KEY = os.getenv("KEY")
    STEAM_ID = os.getenv("STEAM_ID")
    url = 'https://api.steampowered.com/IEconItems_440/GetSchemaItems/v0001/?steamid=' + STEAM_ID + '&key=' + KEY
    response = requests.get(url).json()

    while(response == {}):
        response = requests.get(url).json()

    result = response['result']
    tf2_items = result['items']

    for item in tf2_items:
        if not('craft_class' in item):
            continue

        if item['craft_class'] == 'weapon':
            defindex = item['defindex']
            schema[defindex] = item
    return schema


def get_duplicate_items():
    load_dotenv()
    KEY = os.getenv("KEY")
    STEAM_ID = os.getenv("STEAM_ID")
    url = 'https://api.steampowered.com/IEconItems_440/GetPlayerItems/v0001/?steamid=' + STEAM_ID + '&key=' + KEY #+ '&format=json'
    response = requests.get(url).json()

    while(response == {}):
        response = requests.get(url).json()

    result = response['result']
    tf2_items = result['items']

    # schema {defindex:info}
    schema = get_schema()
    #backpack {defindex:numberOfOccurences}
    backpack = {}
    
    for item in tf2_items:
        defindex = item['defindex']

        if not(defindex in schema):
            continue

        if defindex in backpack:
            backpack[defindex] += 1
        else:
            backpack[defindex] = 1

    duplicates = {}
    for defindex,num in backpack.items():
        if num > 1:
            duplicates[defindex] = num - 1
            #print(schema[defindex]['name'],":",duplicates[defindex])
    return duplicates

get_duplicate_items()


{404: 1,
 528: 1,
 237: 2,
 595: 2,
 406: 4,
 42: 3,
 128: 2,
 163: 1,
 61: 1,
 59: 2,
 131: 5,
 141: 1,
 810: 6,
 305: 3,
 594: 4,
 127: 1,
 349: 1,
 773: 1,
 402: 2,
 751: 3,
 354: 1,
 225: 2,
 220: 2,
 526: 2,
 775: 1,
 228: 1,
 413: 4,
 43: 5,
 41: 3,
 405: 1,
 307: 3,
 35: 3,
 130: 2,
 426: 1,
 411: 5,
 996: 2,
 442: 1,
 448: 2,
 741: 4,
 37: 1,
 40: 1,
 1103: 3,
 482: 2,
 357: 2,
 36: 1,
 232: 4,
 239: 1,
 310: 2,
 230: 4,
 154: 2,
 172: 2,
 155: 5,
 57: 3,
 1098: 5,
 140: 3,
 226: 3,
 215: 1,
 355: 3,
 38: 1,
 312: 3,
 159: 1,
 449: 1,
 752: 4,
 265: 1,
 648: 5,
 214: 3,
 593: 1,
 1104: 7,
 1099: 2,
 222: 1,
 513: 1,
 317: 2,
 401: 3,
 58: 3,
 425: 3,
 589: 2,
 326: 1,
 412: 1,
 171: 1,
 60: 2,
 311: 5,
 231: 4,
 351: 3,
 441: 2,
 153: 1,
 812: 2,
 142: 1,
 304: 1,
 772: 1,
 1101: 2,
 447: 1,
 656: 5,
 811: 1,
 444: 1,
 348: 1,
 997: 1,
 450: 1,
 460: 1,
 1092: 2,
 44: 4,
 133: 1,
 56: 1,
 527: 1}

In [6]:
def sort_by_class():
    duplicates = get_duplicate_items()

    schema = get_schema()

    # items_class  = {class:[defindex]} (list of all items, repeats allowed)
    items_by_class = {}

    # shared_guns= {defindex:[num,[classes]]}
    shared_items = {}

    for defindex,num in duplicates.items():
        classes = schema[defindex]['used_by_classes']

        if len(classes) > 1:
            shared_items[defindex] = [num,classes]
            continue
        
        curr_class = classes[0]
        if not(curr_class in items_by_class):
            items_by_class[curr_class] = []
        
        for i in range(num):
            items_by_class[curr_class].append(defindex)
    
    return items_by_class,shared_items

In [7]:
def start_search():
    # items_class  = {class:[defindex]} (list of all items, repeats allowed)
    # shared_items = {defindex:[num,[classes]]}
    # operations = [(weapon1,weapon2)]
    items_by_class, shared_items = sort_by_class()
    schema = get_schema()

    for defindex,info in shared_items.items():
        num = info[0]
        for i in range(num):
            # give item to first class with odd number of items (or last class)
            classes = info[1]
            num_classes = len(classes)

            for i in range(num_classes):
                curr_class = classes[i]
                weapons_free = items_by_class[curr_class]
                num_weapons_free = len(weapons_free)

                if (num_weapons_free % 2 == 1) or (i == num_classes - 1):
                    items_by_class[curr_class].append(defindex)

    return items_by_class

In [8]:
def search():
    items_by_class = start_search()
    schema = get_schema()

# list of tuplpes [(weapon1,weapon2)]
    operations = []
    gun_names_operations = []

    for game_class,weapons in items_by_class.items():

        while(len(weapons) >= 2):
            weapon1 = weapons.pop()
            weapon2 = weapons.pop()

            operations.append((weapon1,weapon2))
            gun_names_operations.append((schema[weapon1]['name'],schema[weapon2]['name']))

    return operations, gun_names_operations

    
operations, gun_names_operations = search()

print(len(operations))
print(gun_names_operations)

113
[('The B.A.S.E. Jumper', 'The B.A.S.E. Jumper'), ('The Pain Train', 'The Pain Train'), ('The Half-Zatoichi', 'The Half-Zatoichi'), ('The Tide Turner', 'The Tide Turner'), ('Stickybomb Jumper', "The Scotsman's Skullcutter"), ("The Scotsman's Skullcutter", "Nessie's Nine Iron"), ("Nessie's Nine Iron", 'The Loose Cannon'), ('The Loose Cannon', 'The Scottish Resistance'), ('The Scottish Resistance', 'The Ullapool Caber'), ('The Ullapool Caber', 'The Ullapool Caber'), ("Ali Baba's Wee Booties", "The Chargin' Targe"), ("The Chargin' Targe", "The Chargin' Targe"), ("The Chargin' Targe", "The Chargin' Targe"), ('The Splendid Screen', 'The Splendid Screen'), ('The Splendid Screen', 'The Splendid Screen'), ('The Widowmaker', 'The Rescue Ranger'), ('The Gunslinger', 'The Eureka Effect'), ('The Eureka Effect', 'The Wrangler'), ('The Wrangler', 'The Wrangler'), ('The Southern Hospitality', 'The Southern Hospitality'), ('The Southern Hospitality', 'The Southern Hospitality'), ('The Southern Hosp